In [22]:
import Pkg; Pkg.add("Combinatorics"); Pkg.add("Optim"); Pkg.add("Random")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [23]:
begin
	using Combinatorics
	using LinearAlgebra
	using Optim
	using Random
end

In [47]:
begin
	# Función para generar un vector θ que cumpla con los requisitos enunciados
	function rand_vector_sum3(dim::Int=8, total::Float64=3.0)
	    while true
	        # Generate dim-1 sorted uniform random numbers in (0, total)
	        cuts = sort(rand(dim - 1) .* total)
	        # Add 0 and total to make full partition
	        parts = diff([0.0; cuts; total])
	        # Retry if any component ≥ 1
	        if all(x -> x < 1.0, parts)
	            return parts
	        end
	    end
	end

	# Función para perturbar el vector θ y generar θ'
	function perturb_vector(p::Vector{Float64}, delta::Float64=0.01)
	  admisible = false
		d = length(p)

		while !admisible
			p_aux = copy(p)
			# Total amount to shift
		  shift_total = delta * k2  # increase 3 elements by delta
		  shift_per_rest = shift_total / (d - k2)
		  for i in 1:(k2+1)
		      p_aux[i] += delta
		  end
		  for i in (k2+1):d
		      p_aux[i] -= shift_per_rest
		  end
		  # Check if the result is still valid
		  if all(x -> 0.0 ≤ x < 1.0, p_aux) && abs(sum(p_aux) - sum(p)) < 1e-10
			admisible = true
		    return(p_aux)
		  else
		    delta = delta*0.95
		  end
		end
		return p_new
	end
end

perturb_vector (generic function with 2 methods)

In [44]:
p = rand_vector_sum3(9, 5.0)
pp = perturb_vector(p)
println(p)
println(pp)
println(sum(pp))

[0.5145823689564966, 0.8269702931372125, 0.7140866786177555, 0.3409457447101394, 0.9772920322888607, 0.4140938178008846, 0.35287058473036526, 0.2115417217618294, 0.647616757996456]
[0.5145823788984893, 0.8269703030792053, 0.7140866885597483, 0.3409457546521322, 0.9772920422308535, 0.4140938153153864, 0.3528705723028742, 0.21154170933433836, 0.647616745568965]
5.000000009941993


In [60]:
begin
	Random.seed!(1234)
	n2 = 11
	k2 = 5
	M2 = collect(CoolLexCombinations(n2,k2))
	M_bool2 = []

	for A in M2
		a = zeros(n2)
		a[A] .= 1
		push!(M_bool2, a)
	end
end

In [61]:
begin
	function f2(λ, p)
	    suma = 0
		for A in M_bool2
		suma += ℯ^(-dot(λ, A))
		end
		logaritmo = log(suma)
		return dot(λ, p) + logaritmo
	end

	function ∇f2(λ, p)
		suma1 = 0
		for A in M_bool2
			suma1 += ℯ^(-dot(λ, A))
		end
		logaritmo = log(suma1)

		g = zeros(n2)
		for i in 1:n2
			suma = 0
			for A in M_bool2
				if A[i] == 1
					suma += ℯ^(-dot(λ, A))
				end
			end
			g[i] = suma
		end

		return p - g*suma1
	end
end

∇f2 (generic function with 1 method)

In [66]:
begin
	encontrado = false
	intentos = 0
	p2 = zeros(n2)
	p2_prima = zeros(n2)
	λ2_opt = zeros(n2)
	λ2_prima_opt = zeros(n2)

	while !encontrado
		intentos += 1
		println("Intentos: ", intentos)
		# Defino p y p'
		p2 = rand_vector_sum3(n2, float(k2))
		p2_prima = perturb_vector(p2, 0.1)

		# Resuelvo para p y p'
		res_p = optimize(λ -> f2(λ, p2),-50ones(n2),50ones(n2), zeros(n2),SAMIN(),Optim.Options(iterations=100000))
		res_p_prima = optimize(λ -> f2(λ, p2_prima),-50ones(n2),50ones(n2), zeros(n2),SAMIN(),Optim.Options(iterations=100000))
		λ2_opt = Optim.minimizer(res_p)
		λ2_prima_opt = Optim.minimizer(res_p_prima)

		# Calculo las probabilidades P(S={1,2,3}) para p y p'
		z2 = 0    # Cte de normalización para p
		z2_prima = 0     # Cte de normalización para p'
		for A in M_bool2
			z2 += ℯ^(-dot(λ2_opt, A))
			z2_prima += ℯ^(-dot(λ2_prima_opt, A))
		end

		B = zeros(n2)
		B[1:k2] .= 1
		p_B = ℯ^(-dot(λ2_opt, B)) * (1/z2)
		p_B_prima = ℯ^(-dot(λ2_prima_opt, B)) * (1/z2_prima)
		encontrado = p_B > p_B_prima
	end

	println("Contraejemplo encontrado:")
	println("Vector p:", p2)
	println("Vector p':", p2_prima)
end

Intentos: 1
Intentos: 2
Intentos: 3
Contraejemplo encontrado:
Vector p:[0.21351281429982494, 0.9602379765973029, 0.5921405162753985, 0.6880272425962013, 0.46035767968617725, 0.8983347928355991, 0.36936736286530714, 0.12537502556796287, 0.01452579070557114, 0.40628739872076114, 0.27183339984989363]
Vector p':[0.21351281439489858, 0.9602379766923765, 0.5921405163704722, 0.6880272426912749, 0.4603576797812509, 0.8983347928514447, 0.3693673627860791, 0.12537502548873483, 0.014525790626343103, 0.4062873986415331, 0.2718333997706656]


In [63]:
begin
	zz2 = 0    # Cte de normalización para p
	zz2_prima = 0     # Cte de normalización para p'
	for A in M_bool2
		zz2 = zz2 + ℯ^(-dot(λ2_opt, A))
		zz2_prima = zz2_prima + ℯ^(-dot(λ2_prima_opt, A))
	end

	B = zeros(n2)
	B[1:k2] .= 1
	p_B = ℯ^(-dot(λ2_opt, B)) * (1/zz2)
	p_B_prima = ℯ^(-dot(λ2_prima_opt, B)) * (1/zz2_prima)

	m = length(M2)
	probabilidades2 = zeros(m)
	for i in 1:m
		probabilidades2[i] = ℯ^(-dot(λ2_opt, M_bool2[i]))
	end

	probabilidades2 = probabilidades2*(1/zz2)

	println("El vector de probabilidades es ", probabilidades2)
	println("La suma de probabilidades es ", sum(probabilidades2))
end

El vector de probabilidades es [0.0005277185511197429, 0.0001683051053768116, 0.000387860972325606, 0.0005125041528804061, 0.0002571951228472333, 4.466087030827109e-5, 0.0005848945529142814, 0.0013478959505915226, 0.0017810564135566477, 0.0008938054852980846, 4.949968068450884e-5, 0.00011407256028934342, 0.00015073097075853791, 0.00028506109188367756, 0.0006569264314511714, 0.0005680317410287141, 0.00042988401586399464, 0.001309035414546363, 0.000868036611770758, 7.564284176673064e-5, 0.00015520562916517385, 0.00020260984757919612, 0.0004669166292304101, 0.0006169651720684555, 0.0003096178486179784, 1.714689033901299e-5, 3.951519797548827e-5, 5.221382018123757e-5, 9.874631946824699e-5, 0.0002275619827264374, 0.00019676850108542532, 0.00014891356826108337, 0.0004534551817853532, 0.00030069140621627564, 2.6202987469197053e-5, 5.9588939600218376e-5, 0.00013732336877985663, 0.00018145366976526225, 0.0003431635911935239, 0.0007908242821813117, 0.0006838106557273196, 0.0005175050081572677, 0

In [64]:
begin
	marginales2 = zeros(n2)
	for i in 1:n2
		for j in 1:m
			if (M_bool2[j])[i] == 1
				marginales2[i] += probabilidades2[j]
			end
		end
	end

	println("Las marginales dadas por los residuos son ", p2)
	println("\nLas marginales calculadas con la distribución óptima son ", marginales2)
	println("\nLa norma 2 de la diferencia entre las marginales es ", norm(p2-marginales2))
end

Las marginales dadas por los residuos son [0.5191535060187691, 0.29721991417215154, 0.23770455205863972, 0.4007035581910283, 0.8221994147509657, 0.23200825750416554, 0.548039657770317, 0.26991372645343725, 0.2990717663155662, 0.4712324556670473, 0.9027531910979123]

Las marginales calculadas con la distribución óptima son [0.519153521591745, 0.2972199203795345, 0.23770456834474848, 0.400703571177681, 0.8221993988401518, 0.23200826396153068, 0.5480396332812556, 0.26991372879392156, 0.29907175926351726, 0.47123244212535426, 0.9027531922405607]

La norma 2 de la diferencia entre las marginales es 4.300462439544199e-8


In [65]:
begin
	println("Contraejemplo encontrado:")
	println("Vector p :", p2)
	println("Vector p':", p2_prima)
	println("Distribución óptima λ para p:", λ2_opt)
	println("Distribución óptima λ' para p':", λ2_prima_opt)
	println("Probabilidades para B = {1,...,k} según λ y λ':", p_B, ";  ", p_B_prima)
end

Contraejemplo encontrado:
Vector p :[0.5191535060187691, 0.29721991417215154, 0.23770455205863972, 0.4007035581910283, 0.8221994147509657, 0.23200825750416554, 0.548039657770317, 0.26991372645343725, 0.2990717663155662, 0.4712324556670473, 0.9027531910979123]
Vector p':[0.5191535061143779, 0.29721991426776034, 0.23770455215424852, 0.4007035582866371, 0.8221994148465745, 0.23200825752010032, 0.548039657690643, 0.26991372637376326, 0.2990717662358922, 0.4712324555873733, 0.9027531910182383]
Distribución óptima λ para p:[0.844150611124036, 1.6790191320726213, 1.9576809895703011, 1.268207205101645, -0.4825300909464755, 1.9869352698737262, 0.7412821260324378, 1.801431508937376, 1.67099510540162, 1.0137740817169785, -1.1498539861615336]
Distribución óptima λ' para p':[-2.8323911902623613, -1.997522672938303, -1.7188607936128042, -2.4083345504689375, -4.15907194064562, -1.689606447814107, -2.9352598440389994, -1.8751103071701254, -2.0055467402354954, -2.662767753117974, -4.826395777862249]
Pr